Purpose of this notebook is a rough outline of how my script will run before pushing it into a full on python file...Below is the macabe theil class which will perform the macabe theil method on my distillation column.

You want to generate binary VLE data at fixed pressure using an activity coefficient model (NRTL, Wilson, UNIQUAC). That means:

Input: two chemicals, pressure, model, and number of points

Output: 
𝑥
x–
𝑦
y diagram + 
𝑇
T–
𝑥
x–
𝑦
y diagram

So you need:

Define the system (components, constants, correlations).

Choose thermodynamic models (EOS + activity model).

Build a flash solver (FlashVL).

Loop over compositions, run bubble/dew flashes, extract results.

Save data & plot.

this clearly doesnt work...

In [16]:
from thermo import (ChemicalConstantsPackage, CEOSGas, CEOSLiquid, FlashVL, NRTL, PRMIX)
import numpy as np
import matplotlib.pyplot as plt

components = ['n-Hexane', 'Cycloheptane']
constants, correlations = ChemicalConstantsPackage.from_IDs(components)

# Extract critical property arrays for the EOS
Tcs = constants.Tcs
Pcs = constants.Pcs
omegas = constants.omegas

# Initial guess for overall composition (zs must be provided)
zs = [0.5, 0.5]

# Equation of state (Peng–Robinson)
eos = PRMIX(Tcs=Tcs, Pcs=Pcs, omegas=omegas, zs=zs)

# Gas + liquid phases
gasPhase = CEOSGas(eos=eos, HeatCapacityGases=correlations.HeatCapacityGases)
liqPhase = CEOSLiquid(eos=eos, HeatCapacityLiquids=correlations.HeatCapacityLiquids)

# Activity model
activity = NRTL(constants=constants)

# Flash solver (gamma–phi approach)
flasher = FlashVL(constants=constants,
                  correlations=correlations,
                  gas=gasPhase,
                  liquids=[liqPhase],
                  activity=activity)

# Sweep overall compositions
z1 = np.linspace(0, 1, 50)
x, y, T = [], [], []

for z in z1:
    res = flasher.flash(T=323.15, P=101325, zs=[z, 1-z])
    x.append(res.liquid0.zs[0])  # liquid mole fraction of comp 1
    y.append(res.gas.zs[0])      # vapor mole fraction of comp 1
    T.append(res.T)

# x–y diagram
plt.plot(x, y, 'o-')
plt.plot([0, 1], [0, 1], '--')
plt.xlabel("x_hexane (liquid)")
plt.ylabel("y_hexane (vapor)")
plt.title("VLE x–y (NRTL + PR EOS)")
plt.grid(True)
plt.show()

# T–x diagram
plt.plot(x, T, 'o-')
plt.xlabel("x_hexane (liquid)")
plt.ylabel("Temperature [K]")
plt.title("T–x bubble curve")
plt.grid(True)
plt.show()


ValueError: Two specs are required